### To effectively predict daily subway ridership in NYC, we explored a range of predictive models, encompassing both linear and non-linear approaches (RandomForest, RNN), to identify the algorithm that best captures the underlying patterns and maximizes predictive accuracy.


In [204]:
# loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import joblib
import seaborn as sns
import math
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot  
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from joblib import dump 
from sklearn.pipeline import Pipeline
import xgboost as xgb
from catboost import CatBoostRegressor
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [ ]:
# loading and preprocessing data
file_path = './cleaned_daily_ridership_weather_holi_data.csv'
df = pd.read_csv(file_path, parse_dates=['date'])
print('data shape: ', df.shape)
# preparing trian and test datasets with scaled data
X = df.drop(columns=['ridership', 'holidayName', 'date', 'weather_code', 'apparent_temperature_mean']) # dropping apparent_temp_mean because it highly correlates with temperature_2m_mean
y = df['ridership']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# standardizing data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [206]:
# building a dataframe to track the models and perforamnce scores
model_comparison_df = pd.DataFrame(columns=['Model Name', 'RMSE', 'R-squared'])

# Linear Regression

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)
lr_y_pred = lr_model.predict(X_test_scaled)
lr_mse = mean_squared_error(y_test, lr_y_pred)
lr_r2 = r2_score(y_test, lr_y_pred)

print(f"R-squared: {lr_r2:.3f}")
lr_rmse = np.sqrt(lr_mse) 
print(f"RMSE: {lr_rmse:.2f}")
print(f"Mean Squared Error: {lr_mse:.2f}")
print(f"Coefficients: {lr_model.coef_}")
print(f"Intercept: {lr_model.intercept_:.2f}")

# extracting feature importance (coefficients)
lr_feature_importance = pd.Series(lr_model.coef_, index=X.columns).sort_values(ascending=False)

# visualizing feature importance
print("\nFeature Importance:")
print(lr_feature_importance)

# Plot feature importance
lr_feature_importance.plot(kind='bar', title="Feature Importance")
plt.ylabel("Coefficient Value")
plt.xlabel("Feature")
plt.show()

lr_results = {'Model Name': 'Linear Regression',
              'RMSE': lr_rmse,
              'R-squared': lr_r2}
model_comparison_df = pd.concat([model_comparison_df, pd.Series(lr_results).to_frame().T], ignore_index=True)
model_comparison_df.head()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

X_train_ols = sm.add_constant(X_train_scaled_df)
X_test_ols = sm.add_constant(X_test_scaled_df)

# Train the OLS model using statsmodels
ols_model = sm.OLS(y_train, X_train_ols).fit()
y_pred_ols = ols_model.predict(X_test_ols)

mse_ols = mean_squared_error(y_test, y_pred_ols)
r2_ols = r2_score(y_test, y_pred_ols)
rmse_ols = np.sqrt(mse_ols)

print("OLS Model Results:")
print(f"R-squared: {r2_ols:.3f}")
print(f"RMSE: {rmse_ols:.2f}")
print(f"Mean Squared Error: {mse_ols:.2f}")
print("\nModel Summary:")
print(ols_model.summary())

# Extract feature importance (coefficients) and p-values
coefficients = ols_model.params[1:] 
p_values = ols_model.pvalues[1:]    

feature_importance_ols = pd.Series(coefficients, index=X_train_scaled_df.columns).sort_values(ascending=False)
p_values_series = pd.Series(p_values, index=X_train_scaled_df.columns).sort_values()

print("\nFeature Importance (Coefficients):")
print(feature_importance_ols)

print("\nP-values:")
print(p_values_series.apply(lambda x: f"{x:.3f}"))

# Plot feature importance (coefficients)
plt.figure(figsize=(8, 4))
plt.bar(feature_importance_ols.index, feature_importance_ols.values)
plt.xticks(rotation=90)
plt.title("OLS Feature Importance (Coefficients)")
plt.ylabel("Coefficient Value")
plt.xlabel("Feature")
plt.tight_layout()
plt.show()

# Plot p-values
plt.figure(figsize=(8, 4))
plt.bar(p_values_series.index, p_values_series.values)
plt.xticks(rotation=90)
plt.title("OLS Feature P-values")
plt.ylabel("P-value")
plt.xlabel("Feature")
plt.axhline(0.05, color='r', linestyle='--')
plt.yticks(np.arange(0, 1.1, 0.1))
plt.tight_layout()
plt.show()

### We can tell that some of the features have high p-values > 0.05, which might be considered less meaningful from statistical perspective. 
* shortwave_radiation_sum, precipitation_sum, wind_speed_10m_max, weather_categories_Precipitation (Light to Moderate), weather_categories_Clear or Fair Weather, precipitation_hours                                    

# Regularized Regressions: Lasso, Ridge, ElasticNet

In [ ]:
def train_evaluate_regularized_models(X_train, y_train, X_test, y_test, feature_names, existing_df):
    models = {
        "Ridge": Ridge(),
        "Lasso": Lasso(),
        "ElasticNet": ElasticNet()
    }

    param_grids = {
        "Ridge": {'alpha': [0.01, 0.1, 1, 10, 100]}, 'max_iter' : [10000],
        "Lasso": {'alpha': [0.01, 0.1, 1, 10, 100]},
        "ElasticNet": {'alpha': [0.01, 0.1, 1], 'l1_ratio': [0.1, 0.5, 0.9]}
    }
    results_list = []

    for name, model in models.items():
        # Hyperparameter tuning
        grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='r2')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        # Predictions and metrics
        y_pred_train = best_model.predict(X_train)
        y_pred_test = best_model.predict(X_test)
        mse_train = mean_squared_error(y_train, y_pred_train)
        mse_test = mean_squared_error(y_test, y_pred_test)
        r2_train = r2_score(y_train, y_pred_train)
        r2_test = r2_score(y_test, y_pred_test)
        r2 = r2_score(y_test, y_pred)

        print(f"\n{name} Model (Best Params: {grid_search.best_params_}):")
        print(f"Overall R-squared: {r2:.3f}")
        print(f"Training R-squared: {r2_train:.3f}")
        print(f"Testing R-squared: {r2_test:.3f}")
        print(f"Training RMSE: {np.sqrt(mse_train):.2f}")
        print(f"Testing RMSE: {np.sqrt(mse_test):.2f}")

        # Feature importance (coefficients)
        importance = pd.Series(best_model.coef_, index=feature_names).sort_values(ascending=False)
        print("\nFeature Importance (coefficients):")
        print(importance)

        results_list.append({
            'Model Name': name,
            'RMSE': np.sqrt(mse_test),
            'R-squared': r2_test,
            'Best Params': grid_search.best_params_
        })

    # Append all results at once
    updated_df = pd.concat([existing_df, pd.DataFrame(results_list)], ignore_index=True)
    return updated_df

feature_names = X.columns.to_list()
model_comparison_df  = train_evaluate_regularized_models(X_train, y_train, X_test, y_test, feature_names, model_comparison_df)
model_comparison_df.head()


### Not much improvement with Regularized regression. Next, we will train non-linear models to capture complex patterns not easily recognized by linear models.

# Tree Based Models

In [ ]:
def tune_evaluate_tree_models(X_train, y_train, X_test, y_test, feature_names, existing_df):
    models = {
        "Decision Tree": {
            "model": DecisionTreeRegressor(random_state=42),
            "params": {
                'max_depth': [5, 7, 10],  # Limiting depth for interpretability
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'ccp_alpha': [0.01, 0.05, 0.1] # Pruning
            }
        },
        "Random Forest": {
            "model": RandomForestRegressor(random_state=42),
            "params": {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [2, 4, 6]
            }
        },
        "Gradient Boosting": {
            "model": GradientBoostingRegressor(random_state=42),
            "params": {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [3, 4, 5],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [2, 4, 6]
            }
        }
    }
    results_list = [] # initializing list to store model results
    for name, model_info in models.items():
        print(f"\nTuning {name}...")
        grid_search = GridSearchCV(model_info["model"], model_info["params"], cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred) # test mse
        rmse = np.sqrt(mse)

        r2 = r2_score(y_test, y_pred) # test r2

        print(f"\n{name} Model (Tuned):")
        print(f"Best Parameters: {grid_search.best_params_}")
        print(f"Test Root Mean Squared Error: {rmse}")
        print(f"Test Mean Squared Error: {mse}")
        print(f"Test R-squared: {r2}")

        results_list.append({
            'Model Name': name,
            'RMSE': rmse,
            'R-squared': r2,
            'Best Params': grid_search.best_params_
        })

        print(f"\n Feature Importances: {best_model.feature_importances_}")
        plt.figure(figsize=(8, 6))
        plt.barh(feature_names, best_model.feature_importances_, color='skyblue')
        plt.xlabel("Feature Importance")
        plt.ylabel("Features")
        plt.title("Random Forest Feature Importances")
        plt.show()
    updated_df = pd.concat([existing_df, pd.DataFrame(results_list)], ignore_index=True)
    return updated_df

feature_names = list(X.columns)
model_comparison_df = tune_evaluate_tree_models(X_train, y_train, X_test, y_test, feature_names, model_comparison_df)

In [ ]:
model_comparison_df.head(10)

### So far, Random Forest has performed the considering high R-squared of 0.688 and low RMSE of 443,943.

# PCA with Regression and RandomForest

In [ ]:
# PCA + Linear Regression with pipeline 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=9)),
    ('regressor', LinearRegression())
])

pipeline.fit(X_train, y_train)
y_pred_pca_lr = pipeline.predict(X_test)
mse_pca_lr = mean_squared_error(y_test, y_pred_pca_lr)
r2_pca_lr = r2_score(y_test, y_pred_pca_lr)

print(f"\nPCA Regression Model (Test performance):")
print(f"PCA R-squared: {r2_pca_lr}")
print(f"PCA Root Mean Squared Error: {np.sqrt(mse_pca_lr)}")
print(f"\nPCA Mean Squared Error: {mse_pca_lr}")

pca_lr_results = {'Model Name': 'PCA Regression',
              'RMSE': np.sqrt(mse_pca_lr),
              'R-squared': r2_pca_lr,
              'Best Params': 'n_components = 9'}
model_comparison_df = pd.concat([model_comparison_df, pd.Series(pca_lr_results).to_frame().T], ignore_index=True)



In [ ]:
# PCA + Random Forest
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# PCA with Random Forest Regressor in a pipeline
pipeline_pca_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=9)),
    ('regressor', RandomForestRegressor(random_state=42))  # Using Random Forest Regressor
])

pipeline_pca_rf.fit(X_train, y_train)
y_pred_pipeline_pca_rf = pipeline_pca_rf.predict(X_test)
mse_pipeline_pca_rf = mean_squared_error(y_test, y_pred_pipeline_pca_rf)
r2_pipeline_pca_rf = r2_score(y_test, y_pred_pipeline_pca_rf)
rmse_pipeline_pca_rf = np.sqrt(mse_pipeline_pca_rf)

print(f"\nPCA with Random Forest Regression Model (Test performance):")
print(f"PCA + RF R-squared: {r2_pipeline_pca_rf}")
print(f"PCA + RF Root Mean Squared Error: {rmse_pipeline_pca_rf}")
print(f"PCA + RF Mean Squared Error: {mse_pipeline_pca_rf}")

# Comparing with Linear Regression PCA model
print(f"\nPCA Regression Model (Test performance):")
print(f"PCA R-squared: {r2_pipeline_pca_rf}")
print(f"PCA Root Mean Squared Error: {np.sqrt(mse_pipeline_pca_rf)}")
print(f"PCA Mean Squared Error: {mse_pipeline_pca_rf}")

pca_rf_results = {'Model Name': 'PCA RanfomForest',
              'RMSE': np.sqrt(mse_pipeline_pca_rf),
              'R-squared': r2_pipeline_pca_rf,
              'Best Params': 'n_components = 9'}
model_comparison_df = pd.concat([model_comparison_df, pd.Series(pca_rf_results).to_frame().T], ignore_index=True)

In [ ]:
model_comparison_df.head(15)

# Fitting XGBoost and Cat Boost

### XGBoost

In [ ]:
# Hyper parameter tunining using GridSearchCV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating DMatrix for XGBoost for faster computation
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param_grid = {
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 5, 10],
}

grid_search = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror', seed=42),
                           param_grid=param_grid, 
                           scoring='r2', 
                           cv=5, 
                           verbose=1)
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

In [ ]:
# Define parameters for XGBoost
params_best = {
    'objective': 'reg:squarederror',
    'max_depth': 4,
    'learning_rate': 0.01,
    'n_estimators': 500,
    'min_child_weight': 10,
    'subsample': 0.6,
    'colsample_bytree': 1.0,
    'seed': 42
}

xgb_regressor_best = xgb.XGBRegressor(**params_best)
xgb_regressor_best.fit(X_train, y_train)
y_pred_xgboost = xgb_regressor_best.predict(X_test)
r2_xgboost = r2_score(y_test, y_pred_xgboost)
mse_xgboost_reg = mean_squared_error(y_test, y_pred_xgboost)
print(f"R-squared: {r2_xgboost}")
print(f"XGBoostRegressor Root Mean Squared Error: {np.sqrt(mse_xgboost_reg)}")
print(f"XGBoostRegressor Mean Squared Error: {mse_xgboost_reg}")

feature_importance_xgboost = xgb_regressor_best.feature_importances_
print("Feature Importances:", feature_importance_xgboost)

xgb_reg_results = {'Model Name': 'XGBoost Regressor',
              'RMSE': np.sqrt(mse_xgboost_reg),
              'R-squared': r2_xgboost,
              'Best Params': grid_search.best_params_}
model_comparison_df = pd.concat([model_comparison_df, pd.Series(xgb_reg_results).to_frame().T], ignore_index=True)

In [ ]:
model_comparison_df.head(20)

# CATBoost

In [ ]:
categorical_features = [
    X.columns.get_loc("weather_categories_Clear or Fair Weather"),
    X.columns.get_loc("weather_categories_Precipitation (Light to Moderate)"),
    X.columns.get_loc("weather_categories_Severe Weather")
]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the CatBoost regressor
catboost_regressor = CatBoostRegressor(
    iterations=500,          # Number of trees
    learning_rate=0.05,      # Step size for gradient descent
    depth=6,                 # Tree depth (controls complexity)
    # cat_features=categorical_features,  # Indices of categorical features
    loss_function='RMSE',    # Root Mean Square Error for regression
    verbose=100,             # Display training updates every 100 iterations
    random_seed=42           # For reproducibility
)

catboost_regressor.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)
y_pred_cat = catboost_regressor.predict(X_test)
r2_cat = r2_score(y_test, y_pred_cat)
mse_catboost_reg = mean_squared_error(y_test, y_pred_cat)
print(f"R-squared: {r2_cat}")
print(f"catboost_regressor Root Mean Squared Error: {np.sqrt(mse_catboost_reg)}")
print(f"catboost_regressor Mean Squared Error: {mse_catboost_reg}")
print(f"R-squared: {r2_cat}")

# Feature Importance Analysis
feature_importance_catboost = catboost_regressor.get_feature_importance(prettified=True)
print("Feature Importances:")
print(feature_importance_catboost)

catboost_reg_results = {'Model Name': 'CATBoost Regressor',
              'RMSE': np.sqrt(mse_catboost_reg),
              'R-squared': r2_cat,
              'Best Params': {'iterations':500, 'learning_rate':0.05, 'depth':6, 'loss_function':'RMSE', 'verbose':100} }
model_comparison_df = pd.concat([model_comparison_df, pd.Series(catboost_reg_results).to_frame().T], ignore_index=True)


# RNN with LSTM
### We're considering fitting an RNN with LSTM because LSTMs, crucial for capturing long-term dependencies in time-series data, which standard RNNs often struggle with. While linear models can identify trends and correlations, and tree-based models can capture non-linear relationships, both often struggle to inherently understand the sequential nature of temporal data.

In [221]:
# Prepare features and target
X_rnn = df.drop(columns=['ridership', 'holidayName', 'weather_code'])
y = df['ridership'].values.astype(np.float32)

# Extract temporal features
X_rnn['year'] = df['date'].dt.year
X_rnn['month'] = df['date'].dt.month
X_rnn['day'] = df['date'].dt.day
X_rnn['day_of_week'] = df['date'].dt.dayofweek
X_rnn['day_of_week_sin'] = np.sin(2 * np.pi * X_rnn['day_of_week'] / 7)
X_rnn['day_of_week_cos'] = np.cos(2 * np.pi * X_rnn['day_of_week'] / 7)
X_rnn = X_rnn.drop(columns=['date']).values.astype(np.float32)

# Scale features and target
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X_rnn)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

In [ ]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=10, delta=0.01):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Load dataset
file_path = './cleaned_daily_ridership_weather_holi_data.csv'
df = pd.read_csv(file_path, parse_dates=['date'])

# Prepare features and target
X_rnn = df.drop(columns=['ridership', 'holidayName', 'weather_code'])
y = df['ridership'].values.astype(np.float32)

# Extract temporal features
X_rnn['year'] = df['date'].dt.year
X_rnn['month'] = df['date'].dt.month
X_rnn['day'] = df['date'].dt.day
X_rnn['day_of_week'] = df['date'].dt.dayofweek
X_rnn['day_of_week_sin'] = np.sin(2 * np.pi * X_rnn['day_of_week'] / 7)
X_rnn['day_of_week_cos'] = np.cos(2 * np.pi * X_rnn['day_of_week'] / 7)
X_rnn = X_rnn.drop(columns=['date']).values.astype(np.float32)

# Scale features and target
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X_rnn)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Chronological train-test split
test_size = 0.2
split_idx = int(len(X_scaled) * (1 - test_size))
X_train, X_test = X_scaled[:split_idx], X_scaled[split_idx:]
y_train, y_test = y_scaled[:split_idx], y_scaled[split_idx:]

# Chronological validation split
val_size = 0.2
val_split_idx = int(len(X_train) * (1 - val_size))
X_train_final, X_val = X_train[:val_split_idx], X_train[val_split_idx:]
y_train_final, y_val = y_train[:val_split_idx], y_train[val_split_idx:]

# Custom Dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_length):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.seq_length], 
                self.y[idx+self.seq_length])

# LSTM Model with Modified Architecture
class RidershipLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(RidershipLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])  # Take the last hidden state
        out = torch.relu(self.fc1(out))
        return self.fc2(out)

# Parameters
seq_length = 30
input_size = X_scaled.shape[1]
hidden_size = 128
num_layers = 2
output_size = 1
batch_size = 32
n_epochs = 100

# Datasets and DataLoaders
train_dataset = TimeSeriesDataset(X_train_final, y_train_final, seq_length)
val_dataset = TimeSeriesDataset(X_val, y_val, seq_length)
test_dataset = TimeSeriesDataset(X_test, y_test, seq_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss, optimizer
model = RidershipLSTM(input_size, hidden_size, num_layers, output_size, dropout=0.3)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
early_stopping = EarlyStopping(patience=10, delta=0.01)

# Training loop with early stopping
train_losses = []
val_losses = []

for epoch in tqdm(range(n_epochs), desc="Training"):
    model.train()
    epoch_train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch.unsqueeze(1))
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    
    # Validation
    model.eval()
    epoch_val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            epoch_val_loss += criterion(outputs, y_batch.unsqueeze(1)).item()
    
    avg_train_loss = epoch_train_loss / len(train_loader)
    avg_val_loss = epoch_val_loss / len(val_loader)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    
    scheduler.step(avg_val_loss)
    early_stopping(avg_val_loss)
    
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break

# Plot Training and Validation Loss
plt.figure(figsize=(12, 6))
plt.plot(train_losses, label='Training Loss', color='blue', linewidth=2)
plt.plot(val_losses, label='Validation Loss', color='red', linewidth=2)
plt.title("Training & Validation Loss Curves", fontsize=14)
plt.xlabel("Epochs", fontsize=12)
plt.ylabel("MSE Loss", fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Evaluate on Test Set
model.eval()
test_preds = []
test_true = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch)
        test_preds.extend(preds.squeeze().numpy())
        test_true.extend(y_batch.numpy())

# Inverse scaling
test_preds_rescaled = scaler_y.inverse_transform(np.array(test_preds).reshape(-1, 1))
test_true_rescaled = scaler_y.inverse_transform(np.array(test_true).reshape(-1, 1))

# Calculate metrics
final_rmse = np.sqrt(mean_squared_error(test_true_rescaled, test_preds_rescaled))
final_r2 = r2_score(test_true_rescaled, test_preds_rescaled)

print(f"\nFINAL MODEL TEST SET PERFORMANCE:")
print(f"RMSE: {final_rmse:.2f}")
print(f"R²: {final_r2:.4f}")

RNN_LSTM_reg_results = {'Model Name': 'RNN LSTM',
              'RMSE': final_rmse,
              'R-squared': final_r2,
              'Best Params': {'seq_length': 30, 'hidden_size': 128, 'num_layer' : 2, 'output_size': 1,'batch_size':32,'n_epochs':100}}
model_comparison_df = pd.concat([model_comparison_df, pd.Series(RNN_LSTM_reg_results).to_frame().T], ignore_index=True)

# Plot Predictions vs Actual
plt.figure(figsize=(14, 6))
plt.plot(test_true_rescaled, label='Actual', color='blue', alpha=0.7, linewidth=2)
plt.plot(test_preds_rescaled, label='Predicted', color='red', alpha=0.7, linewidth=1.5)
plt.title(f"Test Set Performance\nRMSE: {final_rmse:.2f}, R²: {final_r2:.4f}", fontsize=14)
plt.xlabel("Time Step", fontsize=12)
plt.ylabel("Ridership", fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Residual Plot
residuals = test_true_rescaled - test_preds_rescaled
plt.figure(figsize=(12, 6))
plt.scatter(test_preds_rescaled, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.title("Residual Plot", fontsize=14)
plt.xlabel("Predicted Values", fontsize=12)
plt.ylabel("Residuals (Actual - Predicted)", fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

# # Save the final model
# torch.save(model.state_dict(), 'ridership_lstm_final.pth')
# print("Final model saved to ridership_lstm_final.pth")

In [ ]:
model_comparison_df.head(30)

# Prediction test on synthetic data

In [ ]:
import datetime

new_date = datetime.date(2025, 7, 1)  # Hypothetical future date
day_of_week = new_date.weekday()

synthetic_row = {

    'temperature_2m_mean': 70, 
    'temp_2m_range': 10,
    'apparent_temperature_mean': 71,
    'apparent_temp_range': 9.229394,
    'daylight_duration': 55520,
    'sunshine_duration':27307.883, 
    'precipitation_sum': 0.003937,
    'snowfall_sum': 0,
    'precipitation_hours': 1,
    'wind_speed_10m_max': 11.887713,
    'wind_direction_10m_dominant': 113.585045,
    'shortwave_radiation_sum': 15.16,
    'weekend': 1,
    'is_holiday': 0, 
    'weather_categories_Clear or Fair Weather': 0,
    'weather_categories_Precipitation (Light to Moderate)': 1,
    'weather_categories_Severe Weather': 0,
    # Add temporal features
    'year': new_date.year,
    'month': new_date.month,
    'day': new_date.day,
    'day_of_week': day_of_week,
    'day_of_week_sin': np.sin(2 * np.pi * day_of_week / 7),
    'day_of_week_cos': np.cos(2 * np.pi * day_of_week / 7)
}

# Convert to DataFrame and scale the features
synthetic_df = pd.DataFrame([synthetic_row])
synthetic_scaled = scaler_X.transform(synthetic_df)

# Combine with the last 29 days from the test set to form a sequence
last_29_days = X_test[-(seq_length - 1):]  # Use the last 29 rows from the test data
synthetic_sequence = np.vstack([last_29_days, synthetic_scaled]).astype(np.float32)

# Convert to tensor for the LSTM model
synthetic_sequence_tensor = torch.tensor(synthetic_sequence).unsqueeze(0)  

# Make the prediction
model.eval()
with torch.no_grad():
    synthetic_prediction_scaled = model(synthetic_sequence_tensor).numpy()

# Rescale the prediction back to the original scale
synthetic_prediction = scaler_y.inverse_transform(synthetic_prediction_scaled)

print(f"Predicted Ridership for {new_date}: {synthetic_prediction[0][0]:.2f}")


##  Use of temporal features such as day of the week yielded better result for RNN + LSTM. Let's explore using such features on our Linear and Tree based models too.

In [ ]:
# Prepare features and target
X_temporal_features = df.drop(columns=['ridership', 'holidayName', 'weather_code', 'apparent_temperature_mean', 'precipitation_hours'])
y = df['ridership'].values.astype(np.float32)

# Extract temporal features
# X_temporal_features['year'] = df['date'].dt.year
X_temporal_features['month'] = df['date'].dt.month
X_temporal_features['day'] = df['date'].dt.day
X_temporal_features['day_of_week'] = df['date'].dt.dayofweek
X_temporal_features['day_of_week_sin'] = np.sin(2 * np.pi * X_temporal_features['day_of_week'] / 7)
X_temporal_features['day_of_week_cos'] = np.cos(2 * np.pi * X_temporal_features['day_of_week'] / 7)
# X_temporal_features = X_temporal_features.drop(columns=['date']).values.astype(np.float32)
X_temporal_features = X_temporal_features.drop(columns=['date']) #.values.astype(np.float32)
# standardizing data
scaler = StandardScaler()
X_scaled_temoral_features = scaler.fit_transform(X_temporal_features)

# Split data
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_scaled_temoral_features, y, test_size=0.3, random_state=42)
X_temporal_features.head()

# Save fitted scaler, which will be helpful during predictin scoring
scaler_filename = "./scaler_temporal_features.pkl"
joblib.dump(scaler, scaler_filename)
print(f"Scaler for temporal features saved to: {scaler_filename}")

In [ ]:
# Training linear model with temporal features
# standardizing data
scaler_lr = StandardScaler()
x_temp_feat_lr = X_temporal_features.drop(columns=['month', 'day', 'day_of_week_sin', 'day_of_week_cos'])
X_scaled_temoral_lr = scaler_lr.fit_transform(x_temp_feat_lr)

# Split data
X_train_tf_lr, X_test_tf_lr, y_train_tf_lr, y_test_tf_lr = train_test_split(X_scaled_temoral_lr, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train_tf_lr, y_train_tf_lr)
y_pred_lr = model.predict(X_test_tf_lr)
mse_lr = mean_squared_error(y_test_tf_lr, y_pred_lr)
r2_lr = r2_score(y_test_tf_lr, y_pred_lr)

# joblib.dump(model, './linear_regression_model.pkl')

print(f"R-squared: {r2_lr:.3f}")
rmse_lr = np.sqrt(mse_lr) 
print(f"RMSE: {rmse_lr:.2f}")
print(f"Mean Squared Error: {mse_lr:.2f}")
print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_:.2f}")
results_list = []
results_list.append({
    'Model Name': f'Linear Regression + temporal_features',
    'RMSE': rmse_lr,
    'R-squared': r2_lr,
    'Best Params': None
})

# Append all results at once
model_comparison_df = pd.concat([model_comparison_df, pd.DataFrame(results_list)], ignore_index=True)

# extracting feature importance (coefficients)
feature_importance = pd.Series(model.coef_, index=x_temp_feat_lr.columns).sort_values(ascending=False)

# visualizing feature importance
print("\nFeature Importance:")
print(feature_importance)

# Plot feature importance
feature_importance.plot(kind='bar', title="Feature Importance")
plt.ylabel("Coefficient Value")
plt.xlabel("Feature")
plt.show()

### Not much difference. Temporal elements mightly add more multicollinearity since month, day of week, day, day of week sin/cos, are related.

In [ ]:
# Training Linear Regression with OLS for P-Values and better interpretations

# Convert scaled arrays back to DataFrames with original column names
X_train_scaled_df = pd.DataFrame(X_train_tf_lr, columns=x_temp_feat_lr.columns)
X_test_scaled_df = pd.DataFrame(X_test_tf_lr, columns=x_temp_feat_lr.columns)

# Add a constant term to the independent variables
X_train_ols = sm.add_constant(X_train_scaled_df)
X_test_ols = sm.add_constant(X_test_scaled_df)

# Train the OLS model using statsmodels
ols_model = sm.OLS(y_train_tf, X_train_ols).fit()

# Make predictions
y_pred_ols = ols_model.predict(X_test_ols)

# Evaluate the model
mse_ols = mean_squared_error(y_test_tf, y_pred_ols)
r2_ols = r2_score(y_test_tf, y_pred_ols)
rmse_ols = np.sqrt(mse_ols)

print("OLS Model Results:")
print(f"R-squared: {r2_ols:.3f}")
print(f"RMSE: {rmse_ols:.2f}")
print(f"Mean Squared Error: {mse_ols:.2f}")
print("\nModel Summary:")
print(ols_model.summary())

# Extract feature importance (coefficients) and p-values
coefficients = ols_model.params[1:]  # Exclude the constant term
p_values = ols_model.pvalues[1:]     # Exclude the constant term

feature_importance_ols = pd.Series(coefficients, index=X_train_scaled_df.columns).sort_values(ascending=False)
p_values_series = pd.Series(p_values, index=X_train_scaled_df.columns).sort_values()

print("\nFeature Importance (Coefficients):")
print(feature_importance_ols)

print("\nP-values:")
print(p_values_series.apply(lambda x: f"{x:.3f}"))

# Plot feature importance (coefficients)
feature_importance_ols.plot(kind='bar', title="OLS Feature Importance (Coefficients)")
plt.ylabel("Coefficient Value")
plt.xlabel("Feature")
plt.show()

# Plot p-values
plt.figure()
p_values_series.plot(kind='bar', title="OLS Feature P-values")
plt.ylabel("P-value")
plt.xlabel("Feature")
plt.axhline(0.05, color='r', linestyle='--') # Significance threshold
plt.yticks(np.arange(0, 1.1, 0.1)) # Ensure y-axis shows values from 0 to 1
plt.show()

In [ ]:
# Regularized Regression + Hyper parameter Tuining
def train_evaluate_regularized_models(X_train, y_train, X_test, y_test, feature_names, existing_df):
    models = {
        "Ridge": Ridge(),
        "Lasso": Lasso(),
        "ElasticNet": ElasticNet()
    }

    param_grids = {
        "Ridge": {'alpha': [0.01, 0.1, 1, 10, 100]}, 'max_iter' : [10000],
        "Lasso": {'alpha': [0.01, 0.1, 1, 10, 100]},
        "ElasticNet": {'alpha': [0.01, 0.1, 1], 'l1_ratio': [0.1, 0.5, 0.9]}
    }
    results_list = []
    for name, model in models.items():
        # Hyperparameter tuning
        grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='r2')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_

        # Predictions and metrics
        y_pred_train = best_model.predict(X_train)
        y_pred_test = best_model.predict(X_test)
        mse_train = mean_squared_error(y_train, y_pred_train)
        mse_test = mean_squared_error(y_test, y_pred_test)
        r2_train = r2_score(y_train, y_pred_train)
        r2_test = r2_score(y_test, y_pred_test)
        r2 = r2_score(y_test, y_pred)

        print(f"\n{name} Model (Best Params: {grid_search.best_params_}):")
        print(f"Overall R-squared: {r2:.3f}")
        print(f"Training R-squared: {r2_train:.3f}")
        print(f"Testing R-squared: {r2_test:.3f}")
        print(f"Training RMSE: {np.sqrt(mse_train):.2f}")
        print(f"Testing RMSE: {np.sqrt(mse_test):.2f}")

        # Feature importance (coefficients)
        importance = pd.Series(best_model.coef_, index=feature_names).sort_values(ascending=False)
        print("\nFeature Importance (coefficients):")
        print(importance)
    
        results_list.append({
            'Model Name': f'{name} + temporal_features',
            'RMSE': np.sqrt(mse_test),
            'R-squared': r2_test,
            'Best Params': grid_search.best_params_
        })

    # Append all results at once
    updated_df = pd.concat([existing_df, pd.DataFrame(results_list)], ignore_index=True)
    return updated_df


feature_names = X_temporal_features.columns.to_list()
model_comparison_df = train_evaluate_regularized_models(X_train_tf, y_train_tf, X_test_tf, y_test_tf, feature_names, model_comparison_df)

In [ ]:
model_comparison_df.head(25)

# Retraining Tree based models using Temporal features such as day of week, month, day, etc.

In [ ]:
# Tree Based models
def tune_evaluate_tree_models(X_train, y_train, X_test, y_test, feature_names, existing_df):
    models = {
        "Decision Tree": {
            "model": DecisionTreeRegressor(random_state=42),
            "params": {
                'max_depth': [5, 7, 10],  # Limiting depth for interpretability
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'ccp_alpha': [0.01, 0.05, 0.1] # Pruning
            }
        },
        "Random Forest": {
            "model": RandomForestRegressor(random_state=42),
            "params": {
                'n_estimators': [50, 100, 200],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [2, 4, 6]
            }
        },
        "Gradient Boosting": {
            "model": GradientBoostingRegressor(random_state=42),
            "params": {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.01, 0.1, 0.2],
                'max_depth': [3, 4, 5],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [2, 4, 6]
            }
        }
    }
    results_list = []
    for name, model_info in models.items():
        print(f"\nTuning {name}...")
        grid_search = GridSearchCV(model_info["model"], model_info["params"], cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)

        r2 = r2_score(y_test, y_pred)

        print(f"\n{name} Model (Tuned):")
        print(f"Best Parameters: {grid_search.best_params_}")
        print(f"Test Root Mean Squared Error: {rmse}")
        print(f"Test Mean Squared Error: {mse}")
        print(f"Test R-squared: {r2}")
        
        results_list.append({
            'Model Name': f'{name} + temporal_features',
            'RMSE': rmse,
            'R-squared': r2,
            'Best Params': grid_search.best_params_
        })

        if name == "Random Forest":
            joblib.dump(best_model, "./best_random_forest_model.pkl")
            print("\nBest RandomForest model saved as 'best_random_forest_model.pkl'")

        print(f"\n Feature Importances: {best_model.feature_importances_}")
        plt.figure(figsize=(8, 6))
        plt.barh(feature_names, best_model.feature_importances_, color='skyblue')
        plt.xlabel("Feature Importance")
        plt.ylabel("Features")
        plt.title("Random Forest Feature Importances")
        plt.show()
    updated_df = pd.concat([existing_df, pd.DataFrame(results_list)], ignore_index=True)
    return updated_df
feature_names = X_temporal_features.columns.to_list()
model_comparison_df = tune_evaluate_tree_models(X_train_tf, y_train_tf, X_test_tf, y_test_tf, feature_names, model_comparison_df)

# CATBoost with additional Temporal features

In [ ]:
categorical_features = [
    X.columns.get_loc("weather_categories_Clear or Fair Weather"),
    X.columns.get_loc("weather_categories_Precipitation (Light to Moderate)"),
    X.columns.get_loc("weather_categories_Severe Weather")
]

# Initialize the CatBoost regressor
catboost_regressor = CatBoostRegressor(
    iterations=500,          # Number of trees
    learning_rate=0.05,      # Step size for gradient descent
    depth=6,                 # Tree depth (controls complexity)
    # cat_features=categorical_features,  # Indices of categorical features
    loss_function='RMSE',    # Root Mean Square Error for regression
    verbose=100,             # Display training updates every 100 iterations
    random_seed=42           # For reproducibility
)


catboost_regressor.fit(X_train_tf, y_train_tf, eval_set=(X_test_tf, y_test_tf), plot=True)
y_pred_catb = catboost_regressor.predict(X_test_tf)
r2_cat = r2_score(y_test_tf, y_pred_catb)
mse_cat = mean_squared_error(y_test_tf, y_pred_catb)
rmse_cat = np.sqrt(mse_cat)
print(f"R-squared: {r2_cat}")
print(f"RMSE: {rmse_cat}")

catboost_reg_results = {'Model Name': 'CATBoost Regressor + temporal_features',
              'RMSE': rmse_cat,
              'R-squared': r2_cat,
              'Best Params': {'iterations':500, 'learning_rate':0.05, 'depth':6, 'loss_function':'RMSE', 'verbose':100} }
model_comparison_df = pd.concat([model_comparison_df, pd.Series(catboost_reg_results).to_frame().T], ignore_index=True)
# Feature Importance Analysis
feature_importance_catboost = catboost_regressor.get_feature_importance(prettified=True)
print("Feature Importances:")
print(feature_importance_catboost)


# FINAL MODELS COMPARISON

In [ ]:
sorted_result_df = model_comparison_df.sort_values(by=['R-squared', 'RMSE', 'Model Name'],
                                            ascending=[False, True, True])

sorted_result_df.to_csv('Models Result Comparison.csv', index=False)
print("Models Result Comparison exported to predicted_data.csv\n")

print("Sorted Model Performance:")
sorted_result_df.head(50)

### Clearly, Randfom Forest with temporal features outperformed other models based on high R-Squared of 0.748 and RMSE 398,865. We will use the Random Forest model for Predction.

## Test runnig the Random forest model Prediction

In [ ]:
X_temporal_features.head()

In [ ]:
# This chunk of code laods the model, preprocess the input data, and spits out the Daily NYC Ridership Prediction
def preprocess_date(df):
    # Preprocesses the date feature by extracting relevant components.
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

    df = df.drop('date', axis=1)
    return df

def load_model_and_scaler(model_path, scaler_path):
    # Loads the trained model and scaler from the specified file paths.
    try:
        with open(model_path, 'rb') as file:
            model = joblib.load(file)
    except FileNotFoundError:
        print(f"Error: Model file not found at {model_path}")
        model = None

    try:
        with open(scaler_path, 'rb') as file:
            scaler = joblib.load(file)
    except FileNotFoundError:
        print(f"Error: Scaler file not found at {scaler_path}")
        scaler = None

    return model, scaler

def preprocess_features(data, scaler):
    # Preprocesses the input features using the loaded scaler.
    if scaler is None:
        print("Error: Scaler not loaded. Cannot preprocess features.")
        return None
    scaled_data = scaler.transform(data)
    scaled_df = pd.DataFrame(scaled_data, columns=data.columns)
    return scaled_df

def make_prediction(model, preprocessed_data):
    # Makes predictions using the loaded model and preprocessed data.
    if model is None:
        print("Error: Model not loaded. Cannot make predictions.")
        return None
    predictions = model.predict(preprocessed_data)
    return predictions

if __name__ == "__main__":
    model_file_path = 'best_random_forest_model.pkl' 
    scaler_file_path = 'scaler_temporal_features.pkl' 

    # Load the model and scaler
    loaded_model, loaded_scaler = load_model_and_scaler(model_file_path, scaler_file_path)

    if loaded_model and loaded_scaler:
        # --- Get input data for prediction ---
        input_data = { # This input data will be coming from the FRONT END USER INTERACTION
            'date': ['2025-04-15'],  
            'temperature_2m_mean': [15.0],
            'daylight_duration': [13.5],
            'sunshine_duration': [10.0],
            'precipitation_sum': [0.0],
            'snowfall_sum': [0.0],
            'wind_speed_10m_max': [5.0],
            'wind_direction_10m_dominant': [270],
            'shortwave_radiation_sum': [20.0],
            'weekend': [1],
            'is_holiday': [1],
            'apparent_temp_range': [10.0],
            'temp_2m_range': [8.0],
            'weather_categories_Clear or Fair Weather': [1],
            'weather_categories_Precipitation (Light to Moderate)': [0],
            'weather_categories_Severe Weather': [0]
            #'month', 'day', 'day_of_week', 'day_of_week_sin', 'day_of_week_cos' will be generated from the 'date' column 
            # using preprocess_date()
            
        }
        input_df = pd.DataFrame(input_data)

        # Preprocess the date temproal feature ---
        input_df_processed_date = preprocess_date(input_df.copy())

        # Select the features that were used for training. Very import to have the same order of columns that was used for RandomForest training
        feature_columns_to_scale = [
            'temperature_2m_mean', 'daylight_duration', 'sunshine_duration',
            'precipitation_sum', 'snowfall_sum', 'wind_speed_10m_max',
            'wind_direction_10m_dominant', 'shortwave_radiation_sum', 'weekend',
            'is_holiday', 'apparent_temp_range', 'temp_2m_range',
            'weather_categories_Clear or Fair Weather',
            'weather_categories_Precipitation (Light to Moderate)',
            'weather_categories_Severe Weather', 'month', 'day', 'day_of_week',
            'day_of_week_sin', 'day_of_week_cos'
        ]

        # Ensure all required columns are present
        if all(col in input_df_processed_date.columns for col in feature_columns_to_scale):
            input_df_for_scaling = input_df_processed_date[feature_columns_to_scale]

            # Preprocess and scale all features using the loaded scaler that was used for model training
            scaled_features_df = preprocess_features(input_df_for_scaling, loaded_scaler)

            if scaled_features_df is not None:
                # Make the prediction 
                prediction = make_prediction(loaded_model, scaled_features_df)

                if prediction is not None:
                    print("Daily NYC Subway Ridership Prediction:", prediction)
                else:
                    print("Prediction failed.")
            else:
                print("Feature scaling failed.")
        else:
            print("Error: Not all required features are present in the input data.")
            missing_features = [col for col in feature_columns_to_scale if col not in input_df_processed_date.columns]
            print("Missing features:", missing_features)

    else:
        print("Failed to load model or scaler.")